# Definition of the project

Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Use the following format:

Question: "Question here"

SPARQLQuery: "SQL Query to run"

SPARQLResult: "Result of the SQLQuery"


Answer: "Final answer here"

Only use the following tables:

In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 97.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.7 MB/s eta 0:00:00


# Libraries

In [2]:
#define the libraries here!

In [3]:
from transformers import LlamaForCausalLM, LlamaTokenizer

In [4]:
import json

# Read Data

In [5]:
with open("questions.json", "r") as fquestions:
  questions = json.load(fquestions)
questions = questions["questions"]

JSONDecodeError: ignored

In [6]:
with open("answers.json","r") as fanswers:
  answers = json.load(fanswers)

JSONDecodeError: ignored

In [7]:
answers = answers["answers"]


NameError: ignored

# Dataset

In [ ]:
# generate a prompt from training data
train_prompts = []
answer_list = []

for i, question in enumerate(questions):
  question_string = question["question"]["string"]
  paraphrased_question = question["paraphrased_question"]["string"]
  query = question["query"]["sparql"]
  entities = question["entities"][0]
  relations = question["relations"][0]
  for answer in answers:
    if "results" in answer["answer"].keys():
      for answer  in answer["answer"]["results"]["bindings"]:
        if "answer" in answer.keys():
          answer_list.append(answer["answer"]["value"])

  answer_str = " ".join(answer_list)
  prompt = "question: " + question_string +"\n paraphrased question: " + paraphrased_question + "\n query: " + query +  "\n entities: " + entities + "\n relations: " + relations + "\n answers: " + answer_str

  train_prompts.append(prompt)



In [108]:
prompts[0]

'question: What are the papers written by the person Wazir Muhammad?paraphrased question: Which papers did the author Wazir Muhammad write?query: SELECT DISTINCT ?answer WHERE { ?answer <https://dblp.org/rdf/schema#authoredBy> <https://dblp.org/pid/211/3355> }entities: <https://dblp.org/pid/211/3355>relations: <https://dblp.org/rdf/schema#authoredBy>answers: '

# Model

In [ ]:
# https://huggingface.co/blog/llama2
